In [48]:
# one off operation
#!pip install azure-identity
#!pip install duckdb
#!pip install deltalake

In [49]:
from   deltalake.writer import write_deltalake, try_get_deltatable
import re
from   urllib.request import urlopen
import os
import requests
from   shutil import unpack_archive
import glob
import duckdb

In [50]:
def download(url,Path,total_files):
    if not os.path.exists(Path):
      os.makedirs(Path, exist_ok=True)
    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique, reverse=True)
    current =  [os.path.basename(x) for x in glob.glob(Path+'*.zip')]
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))[:total_files] 
    print(str(len(files_to_upload)) + ' New File Loaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

In [51]:
def unzip(Source, Destination):
    if not os.path.exists(Destination):
      os.makedirs(Destination, exist_ok=True)
    filelist=[os.path.basename(x) for x in glob.glob(Source+'*.zip')]
    ### checl the unzipped files already
    current = [os.path.basename(x) for x in glob.glob(Destination+'*.CSV')]
    current = [w.replace('.CSV','.zip') for w in current]
    #unzip only the delta
    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))
    print(str(len(files_to_upload)) + ' New File uncompressed')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
        unpack_archive(str(Source+x), str(Destination), 'zip')
      return "done"
    else:
     return "nothing to see here"

In [52]:
def check_delta(Source,Onelake_table,storage_options):
 dt =try_get_deltatable(Onelake_table,storage_options=storage_options)
 if dt is not None:
    xxxx = dt.to_pyarrow_dataset()
    existing_files = duckdb.sql(' select distinct filename as filename from xxxx').df()['filename'].tolist()
 else:
  existing_files=[]
 filelist_csv=[os.path.basename(x) for x in glob.glob(Source+'*.CSV')]
 files_to_upload = list(set(filelist_csv) - set(existing_files))
 files_to_upload = list(dict.fromkeys(files_to_upload))
 files_to_upload_full_Path = [Source + i for i in files_to_upload]
 print(str(len(files_to_upload_full_Path)) + ' New File to ingest into Onelake')
 return files_to_upload_full_Path

In [53]:
def process_csv(files_to_process):
	raw =duckdb.sql(F"""from read_csv({files_to_process},
		Skip=1,header =0,all_varchar=1,
		columns={{
		'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
		'DUID': 'VARCHAR','INTERVENTION': 'VARCHAR','DISPATCHMODE': 'VARCHAR','AGCSTATUS': 'VARCHAR','INITIALMW': 'VARCHAR',
		'TOTALCLEARED': 'VARCHAR','RAMPDOWNRATE': 'VARCHAR','RAMPUPRATE': 'VARCHAR','LOWER5MIN': 'VARCHAR',
		'LOWER60SEC': 'VARCHAR','LOWER6SEC': 'VARCHAR','RAISE5MIN': 'VARCHAR','RAISE60SEC': 'VARCHAR',
		'RAISE6SEC': 'VARCHAR','MARGINAL5MINVALUE': 'VARCHAR','MARGINAL60SECVALUE': 'VARCHAR',
		'MARGINAL6SECVALUE': 'VARCHAR','MARGINALVALUE': 'VARCHAR','VIOLATION5MINDEGREE': 'VARCHAR',
		'VIOLATION60SECDEGREE': 'VARCHAR','VIOLATION6SECDEGREE': 'VARCHAR','VIOLATIONDEGREE': 'VARCHAR',
		'LOWERREG': 'VARCHAR','RAISEREG': 'VARCHAR','AVAILABILITY': 'VARCHAR','RAISE6SECFLAGS': 'VARCHAR',
		'RAISE60SECFLAGS': 'VARCHAR','RAISE5MINFLAGS': 'VARCHAR','RAISEREGFLAGS': 'VARCHAR',
		'LOWER6SECFLAGS': 'VARCHAR','LOWER60SECFLAGS': 'VARCHAR','LOWER5MINFLAGS': 'VARCHAR',
		'LOWERREGFLAGS': 'VARCHAR','RAISEREGAVAILABILITY': 'VARCHAR','RAISEREGENABLEMENTMAX': 'VARCHAR',
		'RAISEREGENABLEMENTMIN': 'VARCHAR','LOWERREGAVAILABILITY': 'VARCHAR','LOWERREGENABLEMENTMAX': 'VARCHAR',
		'LOWERREGENABLEMENTMIN': 'VARCHAR','RAISE6SECACTUALAVAILABILITY': 'VARCHAR',
		'RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
		'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR',
		'LOWER60SECACTUALAVAILABILITY': 'VARCHAR','LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR'
		}},
		filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
		where I='D' and UNIT ='DUNIT' AND VERSION = '3'                  """)

	df =duckdb.sql("""
		select UNIT, DUID,parse_filename(filename) as filename,cast(columns(*exclude(DUID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
		cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
		isoyear (cast (SETTLEMENTDATE as timestamp)) as year  from raw
		""").arrow()
	return df


In [54]:
def get_token():
    try:
        token = notebookutils.credentials.getToken('storage')
    except:
        !pip install -q azure-identity
        from   azure.identity   import ClientSecretCredential
        import configparser
        config = configparser.ConfigParser()
        config.read("C:/KV/variable.ini")
        credential = ClientSecretCredential(
                        client_id     =config.get("myvars", "appId"),
                        client_secret =config.get("myvars", "secret") , 
                        tenant_id     =config.get("myvars", "tenantId")
                        )
        token =       credential.get_token("https://storage.azure.com/.default").token
    return token

# Process Logic

In [55]:
total_files = 10

In [56]:
ZIP           = "/lakehouse/default/Files/0_Source/ARCHIVE/Daily_Reports/"
CSV           = "/lakehouse/default/Files/1_Transform/0/ARCHIVE/Daily_Reports/"
Onelake_table = "abfss://aemodev@onelake.dfs.fabric.microsoft.com/storage.Lakehouse/Tables/scada"
# Download zip files from the web
download("https://nemweb.com.au/Reports/Current/Daily_Reports/",ZIP,total_files)
# Unzip those files
unzip(ZIP,CSV)

10 New File Loaded
10 New File uncompressed


'done'

In [57]:
# Checks which files were already uploaded to Onlake
storage_options= {"bearer_token": get_token(), "use_fabric_endpoint": "true"}
files_to_process = check_delta(CSV,Onelake_table,storage_options)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

10 New File to ingest into Onelake


In [58]:
# Upload only new files
if len(files_to_process) >0:
	df = process_csv(files_to_process)
	write_deltalake(Onelake_table,
	               df,
			       mode="append",
				   partition_by=['year'],
	               storage_options = storage_options)
else:
	print('all data was loaded already')